# Hypothesis testing 

This notebook is based on recipes in https://github.com/galenwilkerson/hypothesis-testing but adds a hands-on, first-principles example and faster numpy-based approaches. We'll test a simple one-sided hypothesis about a thumbs-up rate for AI conversational output.

We'll work through a detailed example: users provide "thumbs‑up" responses to Copilot output. We'll:

1. Compute a p-value from first principles (binomial probability) for a simple one-sided test (H0: p = 0.01 vs H1: p > 0.01).
2. Compare an analytic normal approximation.
3. Show a fast Monte Carlo / numpy simulation approach.

We'll pick a sample size and observed successes that give a p-value near the 1% level (so it's a demonstrably significant result at the 95% level).

In [2]:
# Imports
import math
import numpy as np
import matplotlib.pyplot as plt

np.random.seed(0)
plt.rcParams['figure.dpi'] = 100


## First principles: binomial test for thumbs-up rate

Setup: we have a control (baseline) flight and treatment flight of Copilot. We want to determine whether the treatment flight is likely to yield more thumbs-up responses.
- Null hypothesis H0: the true thumbs-up rate p = 0.01 (baseline)
- Alternative H1: p > 0.01 (one-sided — we want to detect a higher thumbs-up rate)

We collect n observations and count k thumbs-up responses. From first principles, under H0 the number of thumbs-ups is Binomial(n, p0). The one-sided p-value is:

$$
\mathrm{p\text{-}value} = P_{X\sim\mathrm{Binomial}(n,p_0)}(X \ge k)
= \sum_{i=k}^n \binom{n}{i} p_0^{i} (1 - p_0)^{n-i}
$$



In [ ]:
# Example parameters 
p0 = 0.01   # baseline thumbs-up rate under H0
n = 1000    # treatment sample size
k = 18      # observed thumbs-up count (choose to target p ~ 0.01)

# compute exact one-sided p-value P(X >= k | p0) using log pmf to avoid overflow

def log_binom_pmf(n, k, p):
    """
    Compute the natural logarithm of the binomial probability mass function:

        log P(X = k) = log( C(n, k) * p**k * (1-p)**(n-k) )

    This function returns the log PMF (natural log) and uses the log-gamma
    function (math.lgamma) to compute log(C(n,k)) in a numerically stable way,
    which is important for large n.

    Parameters
    ----------
    n : int
        Number of Bernoulli trials.
    k : int
        Number of observed successes.
    p : float
        Success probability (0 < p < 1) under the null hypothesis.

    Returns
    -------
    float
        The natural log of the binomial PMF at k.
    """
    return math.lgamma(n+1) - math.lgamma(k+1) - math.lgamma(n-k+1) + k*math.log(p) + (n-k)*math.log(1-p)

ks = np.arange(k, n+1)
logpmfs = np.array([log_binom_pmf(n, i, p0) for i in ks])
pmfs = np.exp(logpmfs)
p_value_exact = pmfs.sum()

print(f"Observed thumbs-up: {k}/{n} = {k/n:.3%}")
print(f"Exact one-sided p-value P(X >= {k} | p0={p0}) = {p_value_exact:.6e}")

if p_value_exact < 0.05:
    print("Result: reject H0 at 95% significance (one-sided)")
else:
    print("Result: fail to reject H0 at 95% significance (one-sided)")


Observed thumbs-up: 18/1000 = 1.800%
Exact one-sided p-value P(X >= 18 | p0=0.01) = 1.383258e-02
Result: reject H0 at 95% significance (one-sided)


### What the Monte Carlo cell does

This cell runs a Monte Carlo simulation to estimate the one-sided p-value under the null hypothesis H0 (`p = p0`).

- It draws `trials` independent samples from a Binomial(`n`, `p0`) null distribution using `np.random.binomial`.
- It computes the empirical p-value as the fraction of simulated trials with counts ≥ the observed `k` (i.e., an estimate of P(X ≥ k) under H0).
- It prints the estimated p-value and plots a histogram of the simulated null distribution; the observed `k` is marked with a red dashed line to show how extreme the observation is.

Notes:
- Monte Carlo is useful when exact calculations are expensive or when SciPy is not available.
- Monte Carlo sampling error scales roughly as 1/√(trials); increase `trials` for more precise estimates.
- A fixed random seed (set earlier) ensures reproducible simulation results.

In [ ]:
# Normal (Gaussian) approximation with continuity correction
mu = n * p0
sigma = math.sqrt(n * p0 * (1 - p0))

# continuity correction: subtract 0.5 from k
z = (k - 0.5 - mu) / sigma
p_value_normal = 0.5 * math.erfc(z / math.sqrt(2))

print(f"Normal approx (continuity-corrected) one-sided p-value = {p_value_normal:.6e}")

print(f"Parameters: mu={mu:.3f}, sigma={sigma:.3f}, z={z:.3f}")


In [ ]:
# Monte Carlo (numpy) simulation to estimate p-value
trials = 100000
sim = np.random.binomial(n, p0, size=trials)
p_value_mc = (sim >= k).mean()

print(f"Monte Carlo p-value estimate (n_runs={trials}) = {p_value_mc:.6e}")

# quick sanity: show a small histogram of simulated counts
plt.hist(sim, bins=30, alpha=0.6)
plt.axvline(k, color='red', linestyle='--', label=f'observed k={k}')
plt.title('Monte Carlo null distribution (Binomial(n,p0))')
plt.xlabel('number of thumbs-up')
plt.legend()
plt.show()


## Faster / numpy-based approaches

After demonstrating the calculation from first principles, here are faster ways to get the same result:
- Use a vectorized log-PMF approach (compute log PMF for the relevant range and exponentiate), which avoids repeated high-level combinatorics and is numerically stable when implemented with log-gamma.
- Use `np.random.binomial` for a fast Monte Carlo estimate (often the simplest when SciPy is unavailable).
- If you have SciPy available, `scipy.stats.binom.sf(k-1, n, p0)` computes the exact survival function efficiently.

In [ ]:
# Vectorized log-PMF approach (deterministic exact result)
ks_all = np.arange(0, n+1)
logpmfs_all = np.array([math.lgamma(n+1) - math.lgamma(i+1) - math.lgamma(n-i+1) + i*math.log(p0) + (n-i)*math.log(1-p0) for i in ks_all])
pmfs_all = np.exp(logpmfs_all)

# survival function P(X >= k)
p_value_vec = pmfs_all[k:].sum()
print(f"Vectorized exact p-value = {p_value_vec:.6e}")

# sanity check: compare values
print(f"Exact (sum over tail) = {p_value_exact:.6e}")
print(f"Normal approx               = {p_value_normal:.6e}")
print(f"Monte Carlo estimate        = {p_value_mc:.6e}")


## Conclusion & exercises

With the chosen parameters (n=1000, k=18) the one-sided p-value under H0: p=0.01 is well below 0.05 (and near 0.01), so the observed rate would be considered significant at the 95% level.

Exercises:
- Vary `n` and `k` to see how sample size affects detectability.
- Try a two-sided test and compare thresholds.
- Replace the Monte Carlo simulation with `scipy.stats.binom.sf` if SciPy is available and compare runtimes.
- Apply the same workflow to a real dataset: collect N user responses, count successes, and report a p-value with a short write-up.